In [1]:
import os
import lxml.etree as ET
import re


class HwpService:
    def hwp2xml(self, file_path: str):
        exefile="hwp5proc"

        mXml = f'{file_path[:-4]}.xml'
        mHwp = file_path
        command = f'{exefile} xml "{mHwp}" > "{mXml}"'

        try:
            os.system(command)
        except:
            pass

        return mXml

    def set_target_tag(self, xml, search_text: str):
        tree = ET.parse(xml)
        root = tree.getroot()
        elements = list(root.iter())

        target_tag = []
        for item in elements:
            if item.text and search_text in item.text and item.text.startswith(search_text):
                target_tag.append(item)
        
        return target_tag
    
    def set_column_tag(self, target_tag):
        column_tag = None
        
        for item in target_tag.iterancestors():
            if item.tag == "ColumnSet":
                column_tag = item

        return column_tag
    
    def set_table_cell(self, column_tag, target_tag):
        table_tag = []
        current_table_tag = []
        start_collecting = False

        for item in column_tag.iter():
            if item == target_tag:
                start_collecting = True
            elif start_collecting and item.tag == "TableCell":
                data = {
                    'row': item.get('row'),
                    'col': item.get('col'),
                    'text': "".join(elem.text for elem in item.findall(".//Text") if elem.text),
                    'rowspan': item.get('rowspan'),
                    'colspan': item.get('colspan')
                }

                if data['text'] == '':
                    continue
                if len(current_table_tag) != 0 and data['row'] == '0' and data['col'] == '0':
                    table_tag.append(current_table_tag)
                    current_table_tag = [data]
                else:
                    current_table_tag.append(data)

        if current_table_tag:
            table_tag.append(current_table_tag)

        return table_tag

    # 기존에는 row0 ~ row[n]까지 하나의 리스트로 여러 이중 리스트가 적용되어 있었는데 현재는 하나의 리스트로 쭉 이어져있기때문에 에러가 발생함
    # 이건 내일 해결하도록 하자
    def delete_non_target_data(self, table_data):
        target_data_text = ['일자', '계측위치', '진동레벨', '소음레벨']

        target_data = [
            sublist for sublist in table_data
            if any(
                entry['row'] in ['0', '1'] and any(keyword in entry['text'] for keyword in target_data_text)
                for entry in sublist
            )
        ]
        
        return target_data

    

In [2]:
path = '/Users/dobby/Desktop/Workspace/3월 계측보고서(인덕원~동탄1공구 S01).hwp'
search_text = '일자별 계측 현황'

service = HwpService()
xml = service.hwp2xml(path)


undefined UnderlineStyle value: 15
defined name/values: {'SOLID': UnderlineStyle.SOLID, 'DASHED': UnderlineStyle.DASHED, 'DOTTED': UnderlineStyle.DOTTED, 'DASH_DOT': UnderlineStyle.DASH_DOT, 'DASH_DOT_DOT': UnderlineStyle.DASH_DOT_DOT, 'LONG_DASHED': UnderlineStyle.LONG_DASHED, 'LARGE_DOTTED': UnderlineStyle.LARGE_DOTTED, 'DOUBLE': UnderlineStyle.DOUBLE, 'LOWER_WEIGHTED': UnderlineStyle.LOWER_WEIGHTED, 'UPPER_WEIGHTED': UnderlineStyle.UPPER_WEIGHTED, 'MIDDLE_WEIGHTED': UnderlineStyle.MIDDLE_WEIGHTED}
undefined UnderlineStyle value: 15
defined name/values: {'SOLID': UnderlineStyle.SOLID, 'DASHED': UnderlineStyle.DASHED, 'DOTTED': UnderlineStyle.DOTTED, 'DASH_DOT': UnderlineStyle.DASH_DOT, 'DASH_DOT_DOT': UnderlineStyle.DASH_DOT_DOT, 'LONG_DASHED': UnderlineStyle.LONG_DASHED, 'LARGE_DOTTED': UnderlineStyle.LARGE_DOTTED, 'DOUBLE': UnderlineStyle.DOUBLE, 'LOWER_WEIGHTED': UnderlineStyle.LOWER_WEIGHTED, 'UPPER_WEIGHTED': UnderlineStyle.UPPER_WEIGHTED, 'MIDDLE_WEIGHTED': UnderlineStyle.MIDDL

In [3]:
target_tag = service.set_target_tag(xml, search_text)

column_tag = []
for item in target_tag:
    column_tag.append(service.set_column_tag(item))

print(target_tag, column_tag)

[<Element Text at 0x10ccc3e80>] [<Element ColumnSet at 0x10611ab00>]


In [5]:

table_cell = []
for index in range(len(column_tag)):
    table_cell.extend(service.set_table_cell(column_tag[index], target_tag[index]))

table_cell

[[{'row': '0', 'col': '0', 'text': '일 자', 'rowspan': '2', 'colspan': '1'},
  {'row': '0', 'col': '1', 'text': '발파횟수', 'rowspan': '2', 'colspan': '1'},
  {'row': '0', 'col': '2', 'text': '발파시간', 'rowspan': '2', 'colspan': '1'},
  {'row': '0', 'col': '3', 'text': '계측위치', 'rowspan': '2', 'colspan': '1'},
  {'row': '0', 'col': '4', 'text': '관리기준', 'rowspan': '1', 'colspan': '3'},
  {'row': '0', 'col': '7', 'text': '측정결과', 'rowspan': '1', 'colspan': '3'},
  {'row': '1',
   'col': '4',
   'text': '진동속도(cm/s)',
   'rowspan': '1',
   'colspan': '1'},
  {'row': '1',
   'col': '5',
   'text': '진동레벨(dB(V))',
   'rowspan': '1',
   'colspan': '1'},
  {'row': '1',
   'col': '6',
   'text': '소음레벨(dB(A))',
   'rowspan': '1',
   'colspan': '1'},
  {'row': '1',
   'col': '7',
   'text': '진동속도(cm/s)',
   'rowspan': '1',
   'colspan': '1'},
  {'row': '1',
   'col': '8',
   'text': '진동레벨(dB(V))',
   'rowspan': '1',
   'colspan': '1'},
  {'row': '1',
   'col': '9',
   'text': '소음레벨(dB(A))',
   'rowspan': '1

In [6]:
table_list = service.delete_non_target_data(table_cell)

table_list

[[{'row': '0', 'col': '0', 'text': '일 자', 'rowspan': '2', 'colspan': '1'},
  {'row': '0', 'col': '1', 'text': '발파횟수', 'rowspan': '2', 'colspan': '1'},
  {'row': '0', 'col': '2', 'text': '발파시간', 'rowspan': '2', 'colspan': '1'},
  {'row': '0', 'col': '3', 'text': '계측위치', 'rowspan': '2', 'colspan': '1'},
  {'row': '0', 'col': '4', 'text': '관리기준', 'rowspan': '1', 'colspan': '3'},
  {'row': '0', 'col': '7', 'text': '측정결과', 'rowspan': '1', 'colspan': '3'},
  {'row': '1',
   'col': '4',
   'text': '진동속도(cm/s)',
   'rowspan': '1',
   'colspan': '1'},
  {'row': '1',
   'col': '5',
   'text': '진동레벨(dB(V))',
   'rowspan': '1',
   'colspan': '1'},
  {'row': '1',
   'col': '6',
   'text': '소음레벨(dB(A))',
   'rowspan': '1',
   'colspan': '1'},
  {'row': '1',
   'col': '7',
   'text': '진동속도(cm/s)',
   'rowspan': '1',
   'colspan': '1'},
  {'row': '1',
   'col': '8',
   'text': '진동레벨(dB(V))',
   'rowspan': '1',
   'colspan': '1'},
  {'row': '1',
   'col': '9',
   'text': '소음레벨(dB(A))',
   'rowspan': '1

[['일시',
  '발파횟수',
  '시간',
  '장약량(kg)',
  '발파위치',
  '발파진동및소음측정치(max값)',
  '지발당장약량',
  '총장약량',
  'STA',
  '발파진동(cm/s)',
  '진동레벨dB(V)',
  '소음레벨dB(A)',
  '측정위치',
  '3월1일',
  '1회',
  '7:01',
  '1.6-2.0',
  '300.0',
  '7k+771.25~774.25(종점)',
  'N/T',
  'N/T',
  'N/T',
  '국원물산',
  '2회',
  '11:47',
  '1.6-2.0',
  '95.7',
  '5k+172.90~171.65(시점)',
  '0.209',
  '72.51',
  '73.35',
  '풍전빌딩',
  '0.141',
  '71.42',
  '74.79',
  '로얄프라자',
  '계',
  '395.7',
  '3월2일',
  '1회',
  '7:02',
  '1.6-2.0',
  '280.0',
  '7k+854.75~857.25(종점)',
  '0.082',
  '63.14',
  '60.99',
  '국원물산',
  '2회',
  '11:51',
  '0.875-2.0',
  '85.0',
  '5k+127.0(시점기재갱)',
  '0.074',
  '62.82',
  '66.59',
  '풍전빌딩',
  '0.051',
  '62.35',
  '67.68',
  '로얄프라자',
  '3회',
  '18:01',
  '1.6-2.0',
  '273.0',
  '7k+857.25~859.75(종점)',
  '0.103',
  '61.62',
  '66.73',
  '국원물산',
  '계',
  '638.0',
  '3월3일',
  '1회',
  '7:01',
  '1.6-2.0',
  '224.6',
  '7k+859.75~862.25(종점)',
  '0.083',
  '63.22',
  '62.45',
  '국원물산',
  '계',
  '224.6',
  '3월4일',
  